In [10]:
import catboost as cb
import gc
import joblib
import json
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import sys
sys.path.append("../")
sys.path.append("../../")
import warnings
warnings.simplefilter("ignore")
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score, fbeta_score, make_scorer
from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [11]:
from utils.common import (
    sigmoid, pad_column_name
)
from utils.constants import *
from utils.eval_helpers import (
    plot_roc_curves, plot_feature_importance, 
    amex_metric, get_final_metric_df, amex_metric_np, lgb_amex_metric
)
from utils.eda_helpers import (
    plot_missing_proportion_barchart, 
    get_cols
)
from utils.extraction_helpers import read_file
from utils.feature_group import (
    CATEGORY_COLUMNS, CONTINUOUS_COLUMNS, NON_FEATURE_COLUMNS
)

In [12]:
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
from cycler import cycler
from IPython.display import display
from colorama import Fore, Back, Style
plt.rcParams['axes.facecolor'] = '#0057b8' # blue
plt.rcParams['axes.prop_cycle'] = cycler(color=['#ffd700'] +
                                         plt.rcParams['axes.prop_cycle'].by_key()['color'][1:])
plt.rcParams['text.color'] = 'w'

In [13]:
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Read Data

In [14]:
%%time
df_dict = {}
for i in range(5):
    df_dict[i] = read_file(f"../{PROCESSED_DATA_PATH}/v9/validation_fold{i}.parquet")

Shape of data: (82603, 4370)
Shape of data: (82603, 4370)
Shape of data: (82603, 4370)
Shape of data: (82602, 4370)
Shape of data: (82602, 4370)
CPU times: user 33.5 s, sys: 29.8 s, total: 1min 3s
Wall time: 22.4 s


In [15]:
labels = read_file(f"../{RAW_DATA_PATH}/train_labels.csv")
target = labels["target"].values

Shape of data: (458913, 2)


In [16]:
y_dict = {}
for i in range(5):
    y_dict[i] = df_dict[i]["target"].copy()

In [17]:
%%time
for i in range(5):
    df_dict[i] = df_dict[i].drop(columns=NON_FEATURE_COLUMNS + ["target"], errors="ignore")
gc.collect()

CPU times: user 4.71 s, sys: 14.9 s, total: 19.6 s
Wall time: 21.8 s


347

In [19]:
cat_features = get_cols(df_dict[0], CATEGORY_COLUMNS)
print(cat_features)

[]


### Tune Catboost using Optuna (KFold)

In [20]:
# prev_study = joblib.load(f"../catboost_new_exp/optuna_study.pkl")
# tdf = prev_study.trials_dataframe()
# tdf = tdf.loc[~tdf["value"].isnull()].reset_index(drop=True)
# tdf.columns

In [42]:
class AmexMetric:
    @staticmethod
    def calc(y_true, y_pred):
        metric, _, _ = amex_metric(y_true, y_pred)
        return metric
    
    def is_max_optimal(self):
        return True # greater is better

    def evaluate(self, approxes, target, weight):            
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        y_true = np.array(target).astype(int)
        approx = approxes[0]
        score = self.calc(y_true, approx)
        return score, 1

    def get_final_error(self, error, weight):
        return error

In [47]:
def objective(trial):
    param = {
        "colsample_bylevel": trial.suggest_float("colsample_bylevel", 0.03, 0.1, log=True),
        "depth": trial.suggest_int("depth", 6, 10),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered"]),
        "bootstrap_type": trial.suggest_categorical(
            "bootstrap_type", ["MVS"]
        ),
        "n_estimators": trial.suggest_categorical("n_estimators", [1500, 1750, 2000]),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 2, 5, log=True),
        "min_data_in_leaf": trial.suggest_categorical("min_data_in_leaf", [1024, 2048]),
        "learning_rate": trial.suggest_categorical("learning_rate", [0.03, 0.06, 0.1]),
        "max_bin": trial.suggest_categorical("max_bin", [127, 255]),
        "objective": "Logloss",
        "used_ram_limit": "20gb",
        "thread_count": 16,
        "eval_metric": AmexMetric(),
        # "task_type": "GPU",
        "devices": '0:1'
    }

    # if param["bootstrap_type"] == "Bernoulli":
    #     param["subsample"] = trial.suggest_float("subsample", 0.6, 0.85)
    
    print(param)
    train_score_list, val_score_list = [], []
    for fold in range(5):
        X_train = pd.concat([df for idx, df in df_dict.items() if idx != fold], ignore_index=True)
        X_val = df_dict[fold]
        print("X shape: ", X_train.shape, X_val.shape)
        y_train = pd.concat([y_dict[idx] for idx in range(5) if idx != fold], ignore_index=True)
        y_val = y_dict[fold]
        print("Y shape: ", y_train.shape, y_val.shape)
        with warnings.catch_warnings():
            warnings.filterwarnings('ignore', category=UserWarning)
            model = cb.CatBoostClassifier(**param)
            model.fit(
                X_train, 
                y_train, 
                eval_set=[(X_val, y_val)], 
                use_best_model=True,
                metric_period=50,
                verbose_eval=50,
                cat_features=cat_features,
                early_stopping_rounds=100
            )
        y_train_pred = model.predict(X_train, prediction_type='RawFormulaVal')
        train_score, train_g, train_t4 = amex_metric(y_train, y_train_pred)
        train_data, X_train, y_train = None, None, None
        y_val_pred = model.predict(X_val, prediction_type='RawFormulaVal')
        val_score, val_g, val_t4 = amex_metric(y_val, y_val_pred)
        valid_data, X_val, y_val = None, None, None
        train_score_list.append(train_score)
        val_score_list.append(val_score)
        print(f"{Fore.BLUE}{Style.BRIGHT}Fold {fold} | Train Score = {train_score:.5f} ({train_g:.4f}, {train_t4:.4f})")
        print(f"{Fore.GREEN}{Style.BRIGHT}Fold {fold} | Val Score = {val_score:.5f} ({val_g:.4f}, {val_t4:.4f}){Style.RESET_ALL}")
        print(f"Clear cache {gc.collect()}")
        
    return np.mean(val_score_list)

In [48]:
# study = joblib.load(f"{CURRENT_EXP_PATH}/optuna_study.pkl")

In [49]:
study = optuna.create_study(direction="maximize")

[I 2022-08-21 03:38:15,617] A new study created in memory with name: no-name-97d35944-bcfb-4dea-a2ce-95bdf592cd65


In [50]:
study.optimize(objective, n_trials=50)

{'colsample_bylevel': 0.050454698092565815, 'depth': 9, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'n_estimators': 1500, 'l2_leaf_reg': 2.7451289723741215, 'min_data_in_leaf': 2048, 'learning_rate': 0.03, 'max_bin': 63, 'objective': 'Logloss', 'used_ram_limit': '20gb', 'thread_count': 16, 'eval_metric': <__main__.AmexMetric object at 0x7fcbec4a29e0>, 'devices': '0:1'}
X shape:  (330410, 4368) (82603, 4368)
Y shape:  (330410,) (82603,)


0:	learn: 0.6915845	test: 0.6905689	best: 0.6905689 (0)	total: 1.1s	remaining: 27m 33s
50:	learn: 0.7672712	test: 0.7618455	best: 0.7618455 (50)	total: 34.9s	remaining: 16m 31s
100:	learn: 0.7787548	test: 0.7729213	best: 0.7729213 (100)	total: 1m 7s	remaining: 15m 30s
150:	learn: 0.7854660	test: 0.7781240	best: 0.7784973 (147)	total: 1m 39s	remaining: 14m 48s
200:	learn: 0.7894451	test: 0.7814150	best: 0.7817305 (199)	total: 2m 11s	remaining: 14m 12s
250:	learn: 0.7930745	test: 0.7827799	best: 0.7827828 (248)	total: 2m 43s	remaining: 13m 31s
300:	learn: 0.7957016	test: 0.7848815	best: 0.7848815 (300)	total: 3m 13s	remaining: 12m 51s
350:	learn: 0.7984149	test: 0.7849632	best: 0.7856020 (342)	total: 3m 44s	remaining: 12m 15s
400:	learn: 0.8003926	test: 0.7858359	best: 0.7858359 (400)	total: 4m 15s	remaining: 11m 40s
450:	learn: 0.8028003	test: 0.7865907	best: 0.7866292 (447)	total: 4m 46s	remaining: 11m 6s
500:	learn: 0.8056789	test: 0.7872089	best: 0.7872686 (484)	total: 5m 18s	remaini

0:	learn: 0.6889373	test: 0.6837398	best: 0.6837398 (0)	total: 1.13s	remaining: 28m 9s
50:	learn: 0.7670921	test: 0.7651666	best: 0.7651666 (50)	total: 35.6s	remaining: 16m 51s
100:	learn: 0.7785601	test: 0.7742705	best: 0.7742705 (100)	total: 1m 7s	remaining: 15m 33s
150:	learn: 0.7849044	test: 0.7781200	best: 0.7783552 (147)	total: 1m 40s	remaining: 14m 57s



KeyboardInterrupt



In [ ]:
# joblib.dump(study, f"{CURRENT_EXP_PATH}/optuna_study.pkl")

In [62]:
study_df = study.trials_dataframe()

In [63]:
study_df = study_df.loc[study_df["state"] == "COMPLETE"]

In [ ]:
# for col in get_cols(study_df, "params"):
#     sns.scatterplot(data=study_df, x=study_df[col], y=study_df["value"])
#     plt.show()

### Single Experiment

In [60]:
param = {
    "colsample_bylevel": 0.1,
    "depth": 10,
    "boosting_type": "Ordered",
    "bootstrap_type": "MVS",
    "n_estimators": 1000,
    "l2_leaf_reg": 1,
    "min_data_in_leaf": 1024,
    "learning_rate": 0.05,
    "max_bin": 255,
    "objective": "Logloss",
    "used_ram_limit": "20gb",
    "thread_count": 16,
    "random_strength": 1.3,
    "scale_pos_weight": 1.35,
    "eval_metric": AmexMetric(),
    # "grow_policy": "Depthwise",
    # "max_leaves": 90,
    # "task_type": "GPU",
}

In [ ]:
print(param)
train_score_list, val_score_list = [], []
for fold in range(5):
    X_train = pd.concat([df for idx, df in df_dict.items() if idx != fold], ignore_index=True)
    X_val = df_dict[fold]
    print("X shape: ", X_train.shape, X_val.shape)
    y_train = pd.concat([y_dict[idx] for idx in range(5) if idx != fold], ignore_index=True)
    y_val = y_dict[fold]
    print("Y shape: ", y_train.shape, y_val.shape)
    with warnings.catch_warnings():
        warnings.filterwarnings('ignore', category=UserWarning)
        model = cb.CatBoostClassifier(**param)
        model.fit(
            X_train, 
            y_train, 
            eval_set=[(X_val, y_val)], 
            use_best_model=True,
            metric_period=50,
            verbose_eval=50,
            cat_features=cat_features,
            early_stopping_rounds=100
        )
    y_train_pred = model.predict(X_train, prediction_type='RawFormulaVal')
    train_score, train_g, train_t4 = amex_metric(y_train, y_train_pred)
    train_data, X_train, y_train = None, None, None
    y_val_pred = model.predict(X_val, prediction_type='RawFormulaVal')
    val_score, val_g, val_t4 = amex_metric(y_val, y_val_pred)
    valid_data, X_val, y_val = None, None, None
    train_score_list.append(train_score)
    val_score_list.append(val_score)
    print(f"{Fore.BLUE}{Style.BRIGHT}Fold {fold} | Train Score = {train_score:.5f} ({train_g:.4f}, {train_t4:.4f})")
    print(f"{Fore.GREEN}{Style.BRIGHT}Fold {fold} | Val Score = {val_score:.5f} ({val_g:.4f}, {val_t4:.4f}){Style.RESET_ALL}")
    print(f"Clear cache {gc.collect()}")

{'colsample_bylevel': 0.1, 'depth': 10, 'boosting_type': 'Ordered', 'bootstrap_type': 'MVS', 'n_estimators': 1000, 'l2_leaf_reg': 1, 'min_data_in_leaf': 1024, 'learning_rate': 0.05, 'max_bin': 255, 'objective': 'Logloss', 'used_ram_limit': '20gb', 'thread_count': 16, 'random_strength': 1.3, 'scale_pos_weight': 1.35, 'eval_metric': <__main__.AmexMetric object at 0x7fc7efb7fee0>}
X shape:  (330410, 4368) (82603, 4368)
Y shape:  (330410,) (82603,)


0:	learn: 0.6764630	test: 0.6732952	best: 0.6732952 (0)	total: 5.14s	remaining: 1h 25m 39s
50:	learn: 0.7755758	test: 0.7685304	best: 0.7685304 (50)	total: 3m 27s	remaining: 1h 4m 15s
100:	learn: 0.7869917	test: 0.7768175	best: 0.7768175 (100)	total: 6m 42s	remaining: 59m 38s
150:	learn: 0.7927805	test: 0.7816190	best: 0.7816291 (147)	total: 9m 51s	remaining: 55m 24s
200:	learn: 0.7976293	test: 0.7832967	best: 0.7836396 (196)	total: 12m 58s	remaining: 51m 33s
250:	learn: 0.8019940	test: 0.7851387	best: 0.7854065 (248)	total: 16m 8s	remaining: 48m 9s
300:	learn: 0.8066188	test: 0.7859108	best: 0.7860225 (289)	total: 19m 21s	remaining: 44m 57s
350:	learn: 0.8118163	test: 0.7867622	best: 0.7870929 (340)	total: 22m 32s	remaining: 41m 40s
400:	learn: 0.8163097	test: 0.7879584	best: 0.7879584 (400)	total: 25m 41s	remaining: 38m 22s
450:	learn: 0.8201758	test: 0.7882930	best: 0.7885578 (443)	total: 28m 51s	remaining: 35m 7s
500:	learn: 0.8239012	test: 0.7886714	best: 0.7888316 (481)	total: 32

0:	learn: 0.6967810	test: 0.6926070	best: 0.6926070 (0)	total: 5.54s	remaining: 1h 32m 11s
